#### 수수료 (수입,지급)

In [11]:
import os
import re
import pandas as pd
import GladUtil as gd

# -- 수입원장
root_dir = f'e:/글로벌백업(180329)'
ym  = '202302'
dir0  = f'{root_dir}/04.수수료작업/{ym}수수료/'
fileN = f'{ym}_수입원장(G).xlsx'
cols = ['지점','직할지점','사원번호','사원명','수수료계','계약일자','계약종류']
print('reading ',fileN)

df = pd.read_excel(dir0+fileN, sheet_name='전체', usecols=cols, dtype={'계약일자': str})

# -- 지점변환
f = fr'{root_dir}\77.EXCEL\My VBA\KJ정산작업Util_4.0.xlsm'
s = '정산작업'
df1 = pd.read_excel(f, sheet_name=s, skiprows=5, usecols='i,j,l,n,p,r')
d1_brh  = df1.iloc[:,[0,1]].dropna()
d1_team = df1.iloc[:,[2,3]].dropna()
d1_fc   = df1.iloc[:,[4,5]].dropna()
d1 = dict(zip(d1_brh.iloc[:,0]  ,d1_brh.iloc[:,1]))
d2 = dict(zip(d1_team.iloc[:,0] ,d1_team.iloc[:,1]))
d3 = dict(zip(d1_fc.iloc[:,0]   ,d1_fc.iloc[:,1]))

df['지점'].fillna('글래드',inplace=True)
df['변환지점'] = df.apply(lambda x: d1.get(x['지점'],x['지점']),axis=1)
df['변환지점'] = df.apply(lambda x: d2.get(x['직할지점'],x['변환지점']),axis=1)
df['변환지점'] = df.apply(lambda x: d3.get(x['사원번호'],x['변환지점']),axis=1)

# -- 요약
df_sum = df.groupby(['변환지점','계약종류'])['수수료계'].sum().reset_index()
df_sum.rename(columns = {'지점':'원지점','변환지점':'지점'}, inplace=True)
df_sum.loc[:, "지사"] = df_sum.지점.map(lambda x: '지사'  if x in gd.GROUP2 
                                    else ('직영' if x in gd.GROUP1 else '오류'))

df_sum.to_clipboard(index=False)
del d1, d2, d3, d1_brh, d1_team, d1_fc
print('end')

reading  202302_수입원장(G).xlsx


In [12]:
# -- 수수료지급
file_list = os.listdir(dir0)
# print(file_list)
file_list2= [ s for s in file_list if s.startswith("마감실적리스트")]
dfAll = pd.DataFrame()

for fname in file_list2: 
    bojong = ''.join(re.findall('\(([^)]+)',  fname))
    print(bojong,' ', fname)
    if (bojong == '생보' or bojong == '장기'):
          cols = ['본부','지점','월납보험료','수수료H','수수료S','구분']
    elif bojong == '자동차':
          cols = ['본부','지점','보험료',    '수수료H','수수료S','납입']
    else: cols = ['본부','지점','납입보험료','수수료H','수수료S','구분']
    
    df1 = pd.DataFrame()
    df1 = pd.read_excel(f'{dir0}{fname}', index_col=False, usecols=cols)
    df1.columns = ['본부','지점','보험료','수수료H','수수료S','구분']
#     print(df1.dtypes)
#     print(df1.head(1))
    df1 = df1[df1['본부'] == 'GLAD']
    if (bojong == '생보' or bojong == '장기'):
        df1['보험료'] = df1['보험료'] * df1['구분'].str.contains('초회|철회|취소')
    dfT = df1.groupby(['지점'])[['수수료H','수수료S','보험료']].sum().reset_index()
    dfT['계약종류'] = bojong
    dfAll = pd.concat([dfAll,dfT])

dfAll.loc[:, "지사"] = dfAll.지점.map(lambda x: '지사'  if x in gd.GROUP2 
                                    else ('직영' if x in gd.GROUP1 else '오류'))
dfAll.to_clipboard(index=False)
print('end2')

자동차   마감실적리스트(자동차)-202303270600.xlsx
일반   마감실적리스트(일반)-202303270600.xlsx
장기   마감실적리스트(장기)-202303270600.xlsx
생보   마감실적리스트(생보)-202303270600.xlsx
end


#### 시상금

In [19]:
dir1  = f'{root_dir}/04.수수료작업/{ym}시상/'
fileN = f'{ym} KJ시상정산.xlsx'
cols = ['지점','장기법인','생보법인','일반','자동차','장기수입','장기지급','생보수입','생보지급']
print('reading ',fileN)

df3 = pd.read_excel(dir1+fileN, sheet_name='정산', usecols='b,e,h,i,j,k:n',skiprows=22)
df3.columns = cols
df3 = df3.iloc[:df3[df3['지점']=='계'].index[0]].dropna()
df3 = df3[df3['지점'].str.contains('글래드')]
df3['손보시상'] = df3['장기법인'] + df3['일반'] + df3['자동차'] + \
                  df3['장기수입'] - df3['장기지급'] 
df3['생보시상'] = df3['생보법인'] + df3['생보수입'] - df3['생보지급'] 
# df3[['지점','손보시상','생보시상']].to_clipboard(index=False)
df3.to_clipboard(index=False)
print('end3')

reading  202302 KJ시상정산.xlsx
end3


#### 기타수수료

In [20]:
dir2  = f'{root_dir}/04.수수료작업/{ym}수수료/'
fileN = f'기타수수료 지급의뢰-{ym}.xlsx'
cols= ['항목','지점','금액','지급자']

df4 = pd.read_excel(dir2+fileN,usecols='c:f',skiprows=2)
df4.columns = cols
df4 = df4.dropna()
df4.to_clipboard(index=False)
print('end4')

end4


#### 본사공제

In [24]:
import tabula

in_file = fr"{root_dir}\04.수수료작업\202301시상\{ym[:4]}년{ym[4:]}월총괄명세서(GLAD_전체).pdf"
df = tabula.read_pdf(in_file)
dft = df[5]
dd1 = dft.iloc[:,:2]
dd2 = dft.iloc[:,3:]
dd1.insert(0,'항목','기타공제')
dd2.insert(0,'항목','일반공제')

df5 = pd.concat([dd1, dd2.rename(columns={'내역.1':'내역','금액.1':'금액',})]).dropna()
df5['금액'] = df5['금액'].str.replace(',','').astype(float)
# -- 일반공제의 양수 -> 음수로 변환
df5['금액'] = [ x*-1  if y=='일반공제' else x  for(x,y)  in zip(df5['금액'], df5['항목'] )]
del dd2, dd1
df5.insert(0,'년월','202301', allow_duplicates=True)
df5['대상'] = ~df5['내역'].str.contains('총합|본사운영비|리스크')
print('end5')

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: 4월 15, 2023 11:21:55 오전 org.apache.pdfbox.pdmodel.font.PDCIDFontType0 <init>
4월 15, 2023 11:21:58 오전 org.apache.pdfbox.pdmodel.font.PDCIDFontType0 <init>



end5


#### 결과출력

In [33]:
outFile = f'{root_dir}/월별정산/지점손익_{ym}.xlsx'
w_mode = 'a' if os.path.isfile(outFile) else 'w'  #파일있으면 append mode

if '실적월' in df_sum.columns:
    pass
else:
    df_sum.insert(0,"실적월",ym)
    dfAll.insert(0,"실적월",ym)
    df3.insert(0,"실적월",ym)
    df4.insert(0,"실적월",ym)

with pd.ExcelWriter(outFile, mode = w_mode, engine="openpyxl",if_sheet_exists='replace') as writer:
    df_sum.to_excel(writer, sheet_name="수입",index=False)
    dfAll.to_excel(writer, sheet_name="지급",index=False)
    df3.to_excel(writer, sheet_name="시상",index=False)
    df4.to_excel(writer, sheet_name="기타",index=False)
    df5.to_excel(writer, sheet_name="본사공제",index=False)
print('completed')

completed
